In [ ]:
!pip install elementpath
!pip install tqdm
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.8/228.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 15.6 MB/s eta 0:00:00


In [ ]:
  import xml.etree.ElementTree as ET

  from tqdm import tqdm
  from ultralytics.utils.downloads import download
  from pathlib import Path

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import yaml
from pathlib import Path
import requests
import zipfile

def download(urls, dir, exist_ok=True):
    dir.mkdir(parents=True, exist_ok=True)  # Crea el directorio si no existe
    for url in urls:
        filename = url.split('/')[-1]
        file_path = dir / filename

        # Descargar el archivo si no existe o si exist_ok=False
        if not file_path.exists() or not exist_ok:
            print(f"Descargando {filename}...")
            response = requests.get(url, stream=True)
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)

        # Extraer el archivo si es un .zip
        if file_path.suffix == '.zip':
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(dir)
            print(f"Descomprimido {filename}")

# Cargar el archivo VOC.yaml
with open('/content/VOC.yaml', 'r') as f:
    yaml_data = yaml.safe_load(f)

# Configurar la ruta raíz para guardar en "content" de Colab
dir = Path('/content')

# Definir URLs de los archivos para descargar
url = 'https://github.com/ultralytics/assets/releases/download/v0.0.0/'
urls = [
    f'{url}VOCtrainval_06-Nov-2007.zip',  # 446MB, 5012 imágenes
    f'{url}VOCtest_06-Nov-2007.zip',      # 438MB, 4953 imágenes
    f'{url}VOCtrainval_11-May-2012.zip'   # 1.95GB, 17126 imágenes
]

# Llamar a la función download para descargar y descomprimir en /content
download(urls, dir=dir, exist_ok=True)



Descargando VOCtrainval_06-Nov-2007.zip...
Descomprimido VOCtrainval_06-Nov-2007.zip
Descargando VOCtest_06-Nov-2007.zip...
Descomprimido VOCtest_06-Nov-2007.zip
Descargando VOCtrainval_11-May-2012.zip...
Descomprimido VOCtrainval_11-May-2012.zip


In [ ]:
from tqdm import tqdm
import yaml
from pathlib import Path
import xml.etree.ElementTree as ET
import os
import shutil

def convert_label(path, lb_path, year, image_id, yaml_data):
    try:
        # Verificar que el archivo XML existe
        xml_path = path / f'VOC{year}/Annotations/{image_id}.xml'
        if not xml_path.exists():
            print(f"XML no encontrado: {xml_path}")
            return False

        in_file = open(xml_path)
        out_file = open(lb_path, 'w')
        tree = ET.parse(in_file)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        # Verificar que el archivo YAML contiene las clases correctas
        names = list(yaml_data['names'].values())

        # Contador para verificar si se escriben etiquetas
        labels_written = 0

        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls in names and int(obj.find('difficult').text) != 1:
                xmlbox = obj.find('bndbox')
                xmin = float(xmlbox.find('xmin').text)
                xmax = float(xmlbox.find('xmax').text)
                ymin = float(xmlbox.find('ymin').text)
                ymax = float(xmlbox.find('ymax').text)

                # Verificar que las coordenadas son válidas
                if xmin >= xmax or ymin >= ymax:
                    print(f"Coordenadas inválidas en {image_id}: xmin={xmin}, xmax={xmax}, ymin={ymin}, ymax={ymax}")
                    continue

                # Convertir coordenadas
                dw = 1./w
                dh = 1./h
                x = (xmin + xmax)/2.0 - 1
                y = (ymin + ymax)/2.0 - 1
                w_box = xmax - xmin
                h_box = ymax - ymin

                x = x*dw
                w_box = w_box*dw
                y = y*dh
                h_box = h_box*dh

                cls_id = names.index(cls)
                out_file.write(f"{cls_id} {x:.6f} {y:.6f} {w_box:.6f} {h_box:.6f}\n")
                labels_written += 1

        in_file.close()
        out_file.close()

        # Verificar si se escribieron etiquetas
        if labels_written == 0:
            print(f"No se escribieron etiquetas para {image_id}")
            if os.path.exists(lb_path):
                os.remove(lb_path)
            return False

        return True

    except Exception as e:
        print(f"Error procesando {image_id}: {str(e)}")
        return False

# Cargar el archivo YAML
with open('/content/VOC.yaml', 'r') as f:
    yaml_data = yaml.safe_load(f)
    print("Clases disponibles:", yaml_data['names'])

# Establecer la ruta del dataset raíz
dir = Path('/content/VOCdevkit')

# Contador para estadísticas
stats = {'procesados': 0, 'exitosos': 0, 'fallidos': 0}

for year, image_set in ('2012', 'train'), ('2012', 'val'), ('2007', 'train'), ('2007', 'val'), ('2007', 'test'):
    print(f"\nProcesando {year}/{image_set}")

    imgs_path = dir / 'images' / f'{image_set}{year}'
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    list_file = dir / f'VOC{year}/ImageSets/Main/{image_set}.txt'

    if not list_file.exists():
        print(f"Archivo de lista no encontrado: {list_file}")
        continue

    with open(list_file) as f:
        image_ids = f.read().strip().split()

    for id in tqdm(image_ids, desc=f'{image_set}{year}'):
        stats['procesados'] += 1

        img_file = dir / f'VOC{year}/JPEGImages/{id}.jpg'
        lb_path = (lbs_path / img_file.name).with_suffix('.txt')

        if img_file.exists():
            # Copiar en lugar de mover la imagen
            shutil.copy2(img_file, imgs_path / img_file.name)

            if convert_label(dir, lb_path, year, id, yaml_data):
                stats['exitosos'] += 1
            else:
                stats['fallidos'] += 1
        else:
            print(f"Imagen no encontrada: {img_file}")
            stats['fallidos'] += 1

# Imprimir estadísticas finales
print("\nEstadísticas de conversión:")
print(f"Total procesados: {stats['procesados']}")
print(f"Convertidos exitosamente: {stats['exitosos']}")
print(f"Fallidos: {stats['fallidos']}")

# Verificar algunas etiquetas generadas
print("\nVerificando algunas etiquetas generadas:")
for year, image_set in ('2012', 'train'), ('2012', 'val'):
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    if lbs_path.exists():
        label_files = list(lbs_path.glob('*.txt'))
        if label_files:
            sample_file = label_files[0]
            print(f"\nContenido de {sample_file}:")
            with open(sample_file, 'r') as f:
                print(f.read())

Clases disponibles: {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}

Procesando 2012/train


train2012: 100%|██████████| 5717/5717 [00:08<00:00, 658.72it/s]



Procesando 2012/val


val2012: 100%|██████████| 5823/5823 [00:06<00:00, 883.02it/s]



Procesando 2007/train


...


Procesando 2007/val


val2007: 100%|██████████| 2510/2510 [00:04<00:00, 586.67it/s]



Procesando 2007/test


test2007: 100%|██████████| 4952/4952 [00:13<00:00, 365.98it/s]


Estadísticas de conversión:
Total procesados: 21503
Convertidos exitosamente: 21503
Fallidos: 0

Verificando algunas etiquetas generadas:

Contenido de /content/VOCdevkit/labels/train2012/2011_002913.txt:
1 0.384000 0.498498 0.700000 0.996997


Contenido de /content/VOCdevkit/labels/val2012/2010_001032.txt:
11 0.601000 0.413333 0.646000 0.517333



In [ ]:
"""
# Función convert_label actualizada para usar yaml_data en lugar de yaml
def convert_label(path, lb_path, year, image_id, yaml_data):
      names = list(yaml_data['names'].values())  # lista de nombres usando yaml_data
      def convert_box(size, box):
          dw, dh = 1. / size[0], 1. / size[1]
          x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
          return x * dw, y * dh, w * dw, h * dh

      in_file = open(path / f'VOC{year}/Annotations/{image_id}.xml')
      out_file = open(lb_path, 'w')
      tree = ET.parse(in_file)
      root = tree.getroot()
      size = root.find('size')
      w = int(size.find('width').text)
      h = int(size.find('height').text)

      names = list(yaml['names'].values())  # names list
      for obj in root.iter('object'):
          cls = obj.find('name').text
          if cls in names and int(obj.find('difficult').text) != 1:
              xmlbox = obj.find('bndbox')
              bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
              cls_id = names.index(cls)  # class id
              out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + '\n')
"""

'\n# Función convert_label actualizada para usar yaml_data en lugar de yaml\ndef convert_label(path, lb_path, year, image_id, yaml_data):\n      names = list(yaml_data[\'names\'].values())  # lista de nombres usando yaml_data\n      def convert_box(size, box):\n          dw, dh = 1. / size[0], 1. / size[1]\n          x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]\n          return x * dw, y * dh, w * dw, h * dh\n\n      in_file = open(path / f\'VOC{year}/Annotations/{image_id}.xml\')\n      out_file = open(lb_path, \'w\')\n      tree = ET.parse(in_file)\n      root = tree.getroot()\n      size = root.find(\'size\')\n      w = int(size.find(\'width\').text)\n      h = int(size.find(\'height\').text)\n\n      names = list(yaml[\'names\'].values())  # names list\n      for obj in root.iter(\'object\'):\n          cls = obj.find(\'name\').text\n          if cls in names and int(obj.find(\'difficult\').text) != 1:\n              xmlbo

In [ ]:
"""
from tqdm import tqdm
import yaml
from pathlib import Path
import xml.etree.ElementTree as ET

def convert_label(path, lb_path, year, image_id, yaml_data):
    def convert_box(size, box):
        dw, dh = 1. / size[0], 1. / size[1]
        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]
        return x * dw, y * dh, w * dw, h * dh

    in_file = open(path / f'VOC{year}/Annotations/{image_id}.xml')
    out_file = open(lb_path, 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    names = list(yaml_data['names'].values())  # Corregido: usando yaml_data en lugar de yaml
    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls in names and int(obj.find('difficult').text) != 1:
            xmlbox = obj.find('bndbox')
            bb = convert_box((w, h), [float(xmlbox.find(x).text) for x in ('xmin', 'xmax', 'ymin', 'ymax')])
            cls_id = names.index(cls)  # class id
            out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + '\n')

# Cargar el archivo VOC.yaml
with open('/content/VOC.yaml', 'r') as f:
    yaml_data = yaml.safe_load(f)

# Establecer la ruta del dataset raíz
dir = Path('/content/VOCdevkit')

for year, image_set in ('2012', 'train'), ('2012', 'val'), ('2007', 'train'), ('2007', 'val'), ('2007', 'test'):
    imgs_path = dir / 'images' / f'{image_set}{year}'
    lbs_path = dir / 'labels' / f'{image_set}{year}'
    imgs_path.mkdir(exist_ok=True, parents=True)
    lbs_path.mkdir(exist_ok=True, parents=True)

    # Ruta absoluta al archivo de lista de imágenes
    list_file = dir / f'VOC{year}/ImageSets/Main/{image_set}.txt'

    # Verificar si el archivo de lista existe antes de abrirlo
    if not list_file.exists():
        print(f"Archivo de lista no encontrado: {list_file}")
        continue  # Saltar al siguiente conjunto si el archivo no existe

    with open(list_file) as f:
        image_ids = f.read().strip().split()
    for id in tqdm(image_ids, desc=f'{image_set}{year}'):
        img_file = dir / f'VOC{year}/JPEGImages/{id}.jpg'  # ruta de imagen
        lb_path = (lbs_path / img_file.name).with_suffix('.txt')  # ruta de etiqueta nueva

        # Comprobar si el archivo de imagen existe antes de moverlo
        if img_file.exists():
            img_file.rename(imgs_path / img_file.name)  # mover imagen
            convert_label(dir, lb_path, year, id, yaml_data)  # convertir etiquetas al formato YOLO
        else:
            print(f"Archivo de imagen no encontrado: {img_file}")
"""

'\nfrom tqdm import tqdm\nimport yaml\nfrom pathlib import Path\nimport xml.etree.ElementTree as ET\n\ndef convert_label(path, lb_path, year, image_id, yaml_data):\n    def convert_box(size, box):\n        dw, dh = 1. / size[0], 1. / size[1]\n        x, y, w, h = (box[0] + box[1]) / 2.0 - 1, (box[2] + box[3]) / 2.0 - 1, box[1] - box[0], box[3] - box[2]\n        return x * dw, y * dh, w * dw, h * dh\n\n    in_file = open(path / f\'VOC{year}/Annotations/{image_id}.xml\')\n    out_file = open(lb_path, \'w\')\n    tree = ET.parse(in_file)\n    root = tree.getroot()\n    size = root.find(\'size\')\n    w = int(size.find(\'width\').text)\n    h = int(size.find(\'height\').text)\n\n    names = list(yaml_data[\'names\'].values())  # Corregido: usando yaml_data en lugar de yaml\n    for obj in root.iter(\'object\'):\n        cls = obj.find(\'name\').text\n        if cls in names and int(obj.find(\'difficult\').text) != 1:\n            xmlbox = obj.find(\'bndbox\')\n            bb = convert_box(

In [ ]:
# 3.1 Tamaño de elementos para carpeta train
 import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/images/test2007'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 4952


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/images/train2007'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 2501


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/images/train2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5717


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/images/val2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5823


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/images/val2007'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 2510


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/labels/test2007'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 4952


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/labels/train2007'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 2501


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/labels/train2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5717


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/labels/val2007'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 2510


In [ ]:
# 3.1 Tamaño de elementos para carpeta train
import os

# ruta de la carpeta
dir_path = r'/content/VOCdevkit/labels/val2012'
count = 0
# Itera dentro de la carpeta, es un contador
for path in os.listdir(dir_path):
    # verifica si la ruta actual es un arhico
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('Número total de elementos:', count)

Número total de elementos: 5823


In [ ]:
import os

# Directorios de imágenes y etiquetas
train_images_dir = '/content/VOCdevkit/images/train2012'
train_labels_dir = '/content/VOCdevkit/labels/train2012'
val_images_dir = '/content/VOCdevkit/images/val2012'
val_labels_dir = '/content/VOCdevkit/labels/val2012'

# Función para obtener la lista de archivos sin la extensión
def obtener_archivos_sin_extension(directorio, extension):
    archivos = [os.path.splitext(f)[0] for f in os.listdir(directorio) if f.endswith(extension)]
    return set(archivos)

# Obtener los archivos de imágenes y etiquetas (sin extensión)
train_images = obtener_archivos_sin_extension(train_images_dir, '.jpg')
train_labels = obtener_archivos_sin_extension(train_labels_dir, '.txt')
val_images = obtener_archivos_sin_extension(val_images_dir, '.jpg')
val_labels = obtener_archivos_sin_extension(val_labels_dir, '.txt')

# Comparar imágenes y etiquetas en train y val
imagenes_sin_etiquetas_train = train_images - train_labels
imagenes_sin_etiquetas_val = val_images - val_labels

# Mostrar imágenes sin etiquetas sin eliminarlas
print(f'Imágenes sin etiquetas en train: {len(imagenes_sin_etiquetas_train)}')
print(f'Imágenes sin etiquetas en val: {len(imagenes_sin_etiquetas_val)}')

# Eliminar imágenes sin etiquetas en train
if imagenes_sin_etiquetas_train:
    print('Eliminando imágenes sin etiquetas en train:')
    for img in imagenes_sin_etiquetas_train:
        img_path = os.path.join(train_images_dir, img + '.jpg')
        print(f'Eliminando {img_path}')
        os.remove(img_path)  # Eliminar la imagen

# Eliminar imágenes sin etiquetas en val
if imagenes_sin_etiquetas_val:
    print('Eliminando imágenes sin etiquetas en val:')
    for img in imagenes_sin_etiquetas_val:
        img_path = os.path.join(val_images_dir, img + '.jpg')
        print(f'Eliminando {img_path}')
        os.remove(img_path)  # Eliminar la imagen

Imágenes sin etiquetas en train: 0
Imágenes sin etiquetas en val: 0


In [ ]:
import os
import random
import shutil
from pathlib import Path

# Directorios de las imágenes y etiquetas
val_images_dir = Path('/content/VOCdevkit/images/val2012')
val_labels_dir = Path('/content/VOCdevkit/labels/val2012')
new_val_images_dir = Path('/content/VOCdevkit/images/val2012_reduced')
new_val_labels_dir = Path('/content/VOCdevkit/labels/val2012_reduced')

# Crear directorios para las nuevas imágenes y etiquetas si no existen
new_val_images_dir.mkdir(exist_ok=True, parents=True)
new_val_labels_dir.mkdir(exist_ok=True, parents=True)

# Obtener la lista de imágenes y etiquetas en la carpeta de validación
val_images = list(val_images_dir.glob('*.jpg'))  # Asumiendo que las imágenes son .jpg
val_labels = [val_labels_dir / (img.stem + '.txt') for img in val_images]

# Comprobar que la cantidad de imágenes y etiquetas coincidan
assert len(val_images) == len(val_labels), "La cantidad de imágenes y etiquetas no coincide."

# Seleccionar aleatoriamente 1200 imágenes y etiquetas
selected_indices = random.sample(range(len(val_images)), 1200)

# Mover las imágenes y etiquetas seleccionadas a la nueva carpeta
for idx in selected_indices:
    shutil.move(str(val_images[idx]), str(new_val_images_dir / val_images[idx].name))
    shutil.move(str(val_labels[idx]), str(new_val_labels_dir / val_labels[idx].name))

print(f"Se han seleccionado y movido {len(selected_indices)} imágenes y etiquetas al nuevo conjunto de validación.")


Se han seleccionado y movido 1200 imágenes y etiquetas al nuevo conjunto de validación.


In [ ]:
!mkdir dataset

In [ ]:
!mkdir dataset/images
!mkdir dataset/labels

In [ ]:
!mv /content/VOCdevkit/images/train2012 /content/dataset/images

In [ ]:
!mv /content/VOCdevkit/images/val2012_reduced /content/dataset/images

In [ ]:
!mv /content/VOCdevkit/labels/train2012 /content/dataset/labels

In [ ]:
!mv /content/VOCdevkit/labels/val2012_reduced /content/dataset/labels

In [ ]:
import os

# Directorio de las etiquetas
images_dir = '/content/dataset/images/train'

# Directorio donde se guardará el archivo 2017.txt
output_dir = '/content/dataset'

# Nombre del archivo de salida
output_file = 'train.txt'

# Función para obtener las rutas de las imágenes
def get_image_paths(label_dir):
    image_paths = []
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.jpg'):
            # Asumimos que el nombre del archivo de imagen es el mismo que el de la etiqueta, pero con extensión .jpg
            image_name = os.path.splitext(label_file)[0] + '.jpg'
            # Construimos la ruta relativa de la imagen
            image_path = f'./images/train/{image_name}'
            image_paths.append(image_path)
    return image_paths

# Obtener las rutas de las imágenes
image_paths = get_image_paths(images_dir)

# Escribir las rutas en el archivo 2017.txt
output_path = os.path.join(output_dir, output_file)
with open(output_path, 'w') as f:
    for path in image_paths:
        f.write(path + '\n')

print(f"Se ha creado el archivo {output_file} con {len(image_paths)} rutas de imágenes.")

Se ha creado el archivo train.txt con 5717 rutas de imágenes.


In [ ]:
import os

# Directorio de las etiquetas
images_dir = '/content/dataset/images/val'

# Directorio donde se guardará el archivo 2017.txt
output_dir = '/content/dataset'

# Nombre del archivo de salida
output_file = 'val.txt'

# Función para obtener las rutas de las imágenes
def get_image_paths(label_dir):
    image_paths = []
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.jpg'):
            # Asumimos que el nombre del archivo de imagen es el mismo que el de la etiqueta, pero con extensión .jpg
            image_name = os.path.splitext(label_file)[0] + '.jpg'
            # Construimos la ruta relativa de la imagen
            image_path = f'./images/val/{image_name}'
            image_paths.append(image_path)
    return image_paths

# Obtener las rutas de las imágenes
image_paths = get_image_paths(images_dir)

# Escribir las rutas en el archivo 2017.txt
output_path = os.path.join(output_dir, output_file)
with open(output_path, 'w') as f:
    for path in image_paths:
        f.write(path + '\n')

print(f"Se ha creado el archivo {output_file} con {len(image_paths)} rutas de imágenes.")

Se ha creado el archivo val.txt con 1200 rutas de imágenes.


In [1]:
# 4. Entrenar el modelo YOLOv8

!pip install ultralytics
from ultralytics import YOLO

# Cargar el modelo YOLOv8n para clasificación
model = YOLO('yolo11s.pt')
# Entrenar el modelo con CIFAR-10
model.train(data='/content/VOC_FINAL.yaml', epochs=20, imgsz=640)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 6.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 18.4M/18.4M [00:00<00:00, 107MB/s] 


Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/VOC_FINAL.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

RuntimeError: Dataset '/content/VOC_FINAL.yaml' error ❌ '/content/VOC_FINAL.yaml' does not exist